![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FWorking+With+Document+AI&dt=Document+AI+-+Process+Responses.ipynb)

# Document AI - Process Responses
> From the [Working With Document AI](https://github.com/statmike/vertex-ai-mlops/blob/main/Working%20With%20Document%20AI/readme.md) series in the [vertex-ai-mlops](https://github.com/statmike/vertex-ai-mlops/blob/main/readme.md) repository.

Document AI is an API where you interact with processors to extract information from documents.  You enable the API, create an instance of a processor in your project, send in document(s), receive back JSON with the extracted information:

<p align="center" width="100%"><center>
    <img src="../architectures/architectures/images/working with/documentai/readme/high_level.png">
</center></p>

This workflow covers techniques to extract elements from the responses.  A prior workflow covered all the ways to process a document, or many documents, using Python as the client and is the **prerequisite** for this workflow: [Document AI - Process Documents](./Document%20AI%20-%20Process%20Documents.ipynb). 



In [ ]:
# colab, install, setup here

---
## Online Responses

---
## Batch Responses

---
## Extraction: Python

---
## Extraction: BigQuery

---
## Extraction: Document AI Toolbox